In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os


In [3]:
class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(F.softplus(x))


In [4]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.dropout1 = nn.Dropout(0.3)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.dropout2 = nn.Dropout(0.4)

        self.fc1 = nn.Linear(256 * 56 * 56, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.act = Mish()

    def forward(self, x):
        x = self.act(self.conv1(x))
        x = self.pool(self.act(self.conv2(x)))
        x = self.dropout1(x)
        x = self.act(self.conv3(x))
        x = self.pool(self.act(self.conv4(x)))
        x = self.dropout2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


In [6]:
data_dir = 'CaltechTinySplit'  # klasör yapını buraya koy
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_ds = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_ds = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
test_ds = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)
test_loader = DataLoader(test_ds, batch_size=16)


In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    model.train()
    for ep in range(num_epochs):
        total, correct, loss_sum = 0, 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            _, pred = torch.max(out, 1)
            total += y.size(0)
            correct += (pred == y).sum().item()
            loss_sum += loss.item()
        scheduler.step()
        print(f"Epoch {ep+1}/{num_epochs} - Loss: {loss_sum:.4f} - Acc: {100*correct/total:.2f}%")
    

In [8]:
def evaluate(model, loader):
    model.eval()
    y_true, y_pred, y_prob = [], [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            prob = F.softmax(out, dim=1)
            _, pred = torch.max(prob, 1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())
            y_prob.extend(prob.cpu().numpy())
    return y_true, y_pred, np.array(y_prob)


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNNModel(len(train_ds.classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40], gamma=0.3)

train_model(model, criterion, optimizer, scheduler)
torch.save(model.state_dict(), 'cnn_mish_model.pt')


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load('cnn_mish_model.pt'))
y_true, y_pred, y_prob = evaluate(model, test_loader)

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')
try:
    auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
except:
    auc = 'AUC hesaplanamadı'

print(f"Accuracy: {acc:.4f}, F1: {f1:.4f}, AUC: {auc}")

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=train_ds.classes, yticklabels=train_ds.classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
